In [1]:
import os
import pandas as pd
import seaborn as sns

In [13]:
from metagov import at2df 
from metagov.utils import ast_eval, get_unique_col_values

In [3]:
def load_data(overwrite=False):
    """Load Contract Objects and Contract Parameters "Keyword-coded records views from Govbase"
    (or, locally from file if it exists)"""
    
    datapath_objects = 'tmp/airtable_contract_objects_data.csv'
    datapath_params = 'tmp/airtable_contract_parameters_data.csv'
    
    if overwrite or not (os.path.isfile(datapath_objects) and os.path.isfile(datapath_params)):
        # Load from Airtable
        at = at2df.get_airtable()
        kwargs = {'view': 'Keyword-coded records'}
        df_objects = at2df.get_table_as_df(at, 'Contract Objects', kwargs=kwargs)
        df_params = at2df.get_table_as_df(at, 'Contract Parameters', kwargs=kwargs)
        
        # Drop unnecessary colums and load list columns
        df_objects.drop(columns=['notice', 'full_comment', 'param', 'return', 'dev', 'title',
                                 'coding_keyword_search', 'coding_topic_search',
                                 'coding_keyword_search_options', 'coding_topic_search_options',
                                 'url', 'repo_update_datetime', 'repo_version'
                                ], 
                        inplace=True, errors='ignore')
        df_params.drop(columns=['full_comment', 'coding_keyword_search_options_from_object', 
                                'project_from_object', 'type_from_object', 'visibility_from_object',
                                'url'], 
                       inplace=True, errors='ignore')
        
        # Load list columns (and convert always-single-item list to string)
        for col in ['inheritance', 'modifiers', 'values']:
            df_objects[col] = df_objects[col].apply(ast_eval)
        df_params['object_id'] = df_params['object_id'].apply(lambda x: x[0] if isinstance(x, list) else x)
        
        # Save to local file
        df_objects.to_csv(datapath_objects)
        df_params.to_csv(datapath_params)

    else:
        # Load from local file
        df_objects = pd.read_csv(datapath_objects, index_col=0)
        df_params = pd.read_csv(datapath_params, index_col=0)
        
        # Load list columns
        for col in ['hand_coding', 'inheritance', 'modifiers', 'values']:
            df_objects[col] = df_objects[col].apply(ast_eval)
        for col in ['hand_coding_from_object']:
            df_params[col] = df_params[col].apply(ast_eval)

    return {'objects': df_objects, 'parameters': df_params}

In [4]:
data = load_data()
df_objects, df_params = data.values()

In [5]:
df_objects.head()

,contract,type,inheritance,visibility,project,description,repo_url,object_name,contract_parameters,hand_coding,is_interface,object_id,modifiers,values
id,,,,,,,,,,,,,,
recoLQ7q54IdNJEsJ,CommonStorage,ContractDefinition,[DSAuth],,Colony - Colony Network,ignore-file-swc-131\nignore-file-swc-108,https://github.com/JoinColony/colonyNetwork/tr...,CommonStorage,"[recEcLkjBIBM2Y5YT, recVsW2krM7UuagVx, recyFXB...",[membership],0,CommonStorage: CommonStorage (JoinColony/colo...,NaN,NaN
rec918lQeIeyoyxja,ContractRecovery,FunctionDefinition,NaN,public,Colony - Colony Network,NaN,https://github.com/JoinColony/colonyNetwork/tr...,approveExitRecovery,NaN,[membership],0,ContractRecovery: approveExitRecovery (JoinCo...,"[recovery, auth]",[]
recXwa9USIDVfYP12,ContractRecovery,FunctionDefinition,NaN,public,Colony - Colony Network,Can only be called by the root role,https://github.com/JoinColony/colonyNetwork/tr...,setRecoveryRole,[recNXPpVtY9FZ1bzS],[membership],0,ContractRecovery: setRecoveryRole (JoinColony...,"[stoppable, auth]",[]
recmtcdwC95Fbkl3v,ContractRecovery,FunctionDefinition,NaN,public,Colony - Colony Network,Can only be called by the root role,https://github.com/JoinColony/colonyNetwork/tr...,removeRecoveryRole,[recVPDgSaIhAd9GzA],[membership],0,ContractRecovery: removeRecoveryRole (JoinCol...,"[stoppable, auth]",[]
recGhCmYVvyVLn3pL,ContractRecovery,FunctionDefinition,NaN,public,Colony - Colony Network,NaN,https://github.com/JoinColony/colonyNetwork/tr...,numRecoveryRoles,NaN,[membership],0,ContractRecovery: numRecoveryRoles (JoinColon...,[],[]


In [22]:
print(f"Including data from {len(df_objects['project'].unique())} projects:")
print("\n".join(sorted(df_objects['project'].unique())))
print(f"\nFound {len(df_objects.index)} keyword-coded contract objects and {len(df_params.index)} parameters")
print("\nKeywords include:")
print(get_unique_col_values(df_objects, 'hand_coding'))

Including data from 16 projects:
Aragon - Aragon Court
Aragon - Aragon Govern
Aragon - DAO Templates
Aragon - Vocdoni
Colony - Colony Network
Compound - Governor Bravo
DAOstack - Alchemy
Gnosis - Gnosis Safe
Jur - Open Layer
Kleros - ERC-792
Kleros - Kleros
MakerDAO - Maker Protocol
Moloch - Moloch v1
Moloch - Moloch v2
OpenLaw - Tribute
OpenZeppelin - Governance

Found 1072 keyword-coded contract objects and 3819 parameters

Keywords include:
dispute_resolution
election
membership
proposal
reputation
voting
['dispute_resolution', 'election', 'membership', 'proposal', 'reputation', 'voting']


In [6]:
df_params.head()

,parameter_name,contract,object_name,visibility,description,type_category,type,object_id,hand_coding_from_object
id,,,,,,,,,
recJoPGJe4per156W,termRandomness,JurorsRegistry,DraftParams,,Randomness seed to be used for the draft,bytes,bytes32,recKevkh70FBHGVVM,"[dispute_resolution, election]"
recuddMFdFT33eRZU,disputeId,JurorsRegistry,DraftParams,,ID of the dispute being drafted,uint,uint256,recKevkh70FBHGVVM,"[dispute_resolution, election]"
recKtUAJOBvvgHwLg,termId,JurorsRegistry,DraftParams,,Term ID of the dispute's draft term,uint,uint64,recKevkh70FBHGVVM,"[dispute_resolution, election]"
rec5IwiDrths5mfhn,selectedJurors,JurorsRegistry,DraftParams,,Number of jurors already selected for the draft,uint,uint256,recKevkh70FBHGVVM,"[dispute_resolution, election]"
recsMoLogv5hFwhBZ,batchRequestedJurors,JurorsRegistry,DraftParams,,Number of jurors to be selected in the given b...,uint,uint256,recKevkh70FBHGVVM,"[dispute_resolution, election]"
